In [1]:
from collections import *
from random import random
import pprint
import operator
import pickle

In [2]:
class Syllable:
    def __init__(self , text):
        self.__words = text.split()
    
    #Reference for this method : https://gist.github.com/miratcan/9196ae2591b1f34ab645520a767ced17
    def __get_syllables_word(self , word):
        syllables = []

        """
        Aşağıdaki satır gelen kelimenin ünlü harfler 1, ünsüzler 0 olacak
        şekilde desenini çıkarır.
        Örneğin: arabacı -> 1010101, türkiye -> 010010
        """

        bits = ''.join(['1' if l in 'aeıioöuü' else '0' for l in word])

        """
        Aşağıdaki seperators listesi, yakalanacak desenleri ve desen yakalandığında
        kelimenin hangi pozisyondan kesileceğini tanımlıyor.
        Türkçede kelime içinde iki ünlü arasındaki ünsüz, kendinden sonraki
        ünlüyle hece kurar., yani 101 desenini yakaladığımızda kelimeyi
        bulunduğumuz yerden 1 ileri pozisyondan kesmeliyiz. ('101', 1)
        Kelime içinde yan yana gelen iki ünsüzden ilki kendinden önceki ünlüyle,
        ikincisi kendinden sonraki ünlüyle hece kurar. Bu da demek oluyor ki
        1001 desenini yakaladığımızda kelimeyi bulunduğumuz noktadan 2 ileriden
        kesmeliyiz. ('1001', 2),
        Kelime içinde yan yana gelen üç ünsüz harften ilk ikisi kendinden önceki
        ünlüyle, üçüncüsü kendinden sonraki ünlüyle hece kurar. Yani 10001 desenini
        gördüğümüzde kelimeyi bulunduğumuz yerden 3 ileri pozisyondan kesmemiz
        gerek. ('10001', 3)
        """

        seperators = (
            ('101', 1),
            ('1001', 2),
            ('10001', 3)
        )

        index, cut_start_pos = 0, 0

        # index değerini elimizdeki bitler üzerinde yürütmeye başlıyoruz.
        while index < len(bits):

            """
            Elimizdeki her ayırıcıyı (seperator), bits'in index'inci karakterinden
            itibarent tek tek deneyerek yakalamaya çalışıyoruz.
            """

            for seperator_pattern, seperator_cut_pos in seperators:
                if bits[index:].startswith(seperator_pattern):

                    """
                    Yakaladığımızda, en son cut_start posizyonundan, bulunduğumuz
                    pozisyonun serpator_cut_pos kadar ilerisine kadar bölümü alıp
                    syllables sepetine atıyoruz.
                    """

                    syllables.append(word[cut_start_pos:index + seperator_cut_pos])

                    """
                    Index'imiz seperator_cut_pos kadar ilerliyor, ve
                    cut_start_pos'u index'le aynı yapıyoruz.
                    """

                    index += seperator_cut_pos
                    cut_start_pos = index
                    break

            """
            Index ilerliyor, cut_start_pos'da değişiklik yok.
            """

            index += 1

        # Son kalan heceyi elle sepete atıyoruz.
        syllables.append(word[cut_start_pos:])
        return '-'.join(syllables)
    def get_all_syllables(self):
        return [self.__get_syllables_word(word) for word in self.__words]

In [3]:
class NGram:
    
    def __init__(self, filePathOrData , syllableOrChar, isFile):
        self.__type = syllableOrChar.lower()
        self.__model = []
        self.__data = ""
        self.__path = filePathOrData
        self.__readData()
        """if (isFile):
            self.__path = filePathOrData
            self.__readData()
        else:
            self.__path = ""
            self.__data = filePathOrData """   
    
    def __readData(self):
        print("Reading Data...")
        self.__data = open(self.__path).read()
    
    def __process_text(self , text):
 
        text = text.lower()
        #text = text.replace(',', ' ')
        text = text.replace('/', ' ')
        text = text.replace('(', ' ')
        text = text.replace(')', ' ')
        text = text.replace('\'' , ' ')
        text = text.replace('\"' , '')
 
        # Convert text string to a list of words
        return text
    
    def __generate_ngrams_for_syllable(self, words_list, n):
        ngrams_list = list()
        for num in range(0, len(words_list)):
            ngram = ' '.join(words_list[num:num + n])
            ngrams_list.append(ngram)
        print("Processing...")
        return ngrams_list

    def __generate_ngrams_for_char(self , data, n):
        ngrams_list = list()
        for i in range(len(data)):
            ngram = ''.join(data[i:i + n])
            ngrams_list.append(ngram)
        print("Processing...")
        return ngrams_list
    
    def __create_word_all_grams_with_probabilies(self ,data,n, function):
        print("Model is creating...")
        #all_grams = list()
        sorted_grams = list()
        #for i in range(n + 1): # for find the all n grams 0,1,2,3,...n gram into model
        n_grams = function(data, n)
        n_counts = Counter(n_grams)
        n_1_grams = function(data , n - 1)
        n_1_counts = Counter(n_1_grams)
        if(self.__type == "character"):
            probs = {gram : (n_counts[gram] + 1)/(len(n_counts) + n_1_counts[''.join(gram[:-1])]) for gram in n_counts.keys()}
        else:
            probs = {gram : (n_counts[gram] + 1)/(len(n_counts) + n_1_counts[' '.join(gram.split()[:-1])]) for gram in n_counts.keys()}
        
        #sorted_grams.append(sorted(probs.items(), key=lambda kv: kv[1] , reverse=True))
        self.__model.append(probs)
        print("Model is created...")
        return self.__model #, sorted_grams
    
    def __syllable_perplexity(self , data):
       
        result = 1.0
        #print(data)
        tokens = data.split()
        power = len(tokens)
        next_gram = ""
        for i in range(power - n + 1):
            next_gram = ' '.join(tokens[i:i + self.__n])
            if (self.__model[0].get(next_gram) == None):
                result *= 1 / len(self.__model[0])
            else:        
                result *= self.__model[0].get(next_gram)
        result = (1 / result) ** (1 / power)
        return result
        
            
    
    def __char_perp(self , text):
        result = 1.0
        power = len(text)
        for i in range(power - n + 1):
            next_gram = ''.join(text[i:i + self.__n])
            if(self.__model[0].get(next_gram) == None):
                result *= 1 / len(self.__model[0])
            else:
                result *= self.__model[0].get(next_gram)
        return (1 / result) ** (1 / power)
    

    
    """def __char_perplexity(self, data):
        result = 1.0
        power = len(tokens)
        for i in range(power):
            result *= self.__model[i + 1].get(''.join(tokens[:i + 1]))
        result = (1 / result) ** (1 / power)
        return result"""
    
    def save_model(self, name ):
        print("{}-Gram model is saving...".format(self.__n))
        with open(name + '.pkl', 'wb') as f:
            pickle.dump(self, f, pickle.HIGHEST_PROTOCOL)
            
    def load_model(name):
        print("N-Gram model is loading...")
        with open(name + '.pkl', 'rb') as f:
            return pickle.load(f)
        
    def create_NGram(self , n = 3):
        print("{} {}-Gram model is creating...".format(self.__type,n))
        self.__n = n
        
        if(self.__type == "character"):
            return self.__create_word_all_grams_with_probabilies(
                             self.__process_text(self.__data),
                             n,
                             function = self.__generate_ngrams_for_char)
        else:
            data = self.__process_text(self.__data)
            data = data.replace("-" , " ")
            data = data.split()
            return self.__create_word_all_grams_with_probabilies(
                            data,
                            n,
                            function = self.__generate_ngrams_for_syllable)
    
    def perplexity(self , text):
        text = text.lower()
        if(self.__type == "character"):
            return "The perplexity of the character type n-gram for the given text : {}".format(self.__char_perp(text))
        else:
            syllables = Syllable(text).get_all_syllables()
            syllables_text = ' '.join(syllables)
            return "The perplexity of the character type n-gram for the given text : {}".format(self.__syllable_perplexity(syllables_text.replace("-" , " ")))#for syllable
        
    """def probability_of(self , text):
        if(self.__type == "character"):
            return self.__model[len(text)].get(text)
        else:
            return self.__model[len(text.split())].get(text)"""
    
    def getModel_info(self):
        return self.__model

In [ ]:
if __name__ == '__main__':
    
    input_file_char = "data.txt"
    input_file_syllable = "syllables_data.txt"
    model_type1 = "character"
    model_type2 = "syllable"
    n = 4
    
    """model1 = NGram(input_file_char, model_type1)
    
    
    model1.create_NGram(n)
    model1.save_model("character_model1")"""
    
    model2 = NGram(input_file_syllable , model_type2 , True)
    model2.create_NGram(n)
    model2.save_model("syllable_model_with_Laplace")
    
    
    #print("Perplexity of \'{0}\' is : {1}".format(test_text , model.perplexity(test_text)))
    
    #print("Probability of \'{0}\' is : {1}".format(test_text , model.probability_of(test_text)))
    
    #model_file = NGram.load_model("character_model1")
    
    #model_file.new_perp("Eylül ayında yayın saatini arttıran")
    
    #model_file.getModel_info()
    
    
    
    
    
        

Reading Data...
syllable 4-Gram model is creating...
Model is creating...
Processing...


In [ ]:
model_file = NGram.load_model("syllable_model1")
model_file.getModel_info()

In [ ]:
model_file.perplexity("Yeşil renkli olarak görülebilirler")

In [ ]:
input_file_syllable = "turkish_file.txt"
model_type1 = "character"
model_type2 = "syllable"
n = 4


#model2 = NGram(' '.join(Syllable(open(input_file_syllable).read()).get_all_syllables()) , model_type2 , False)
model2 = NGram(input_file_syllable , model_type1 , True)


model2.create_NGram(n)

In [ ]:
print(model2.perplexity("kalabalık"))